In [1]:
import numpy as np
import cv2
import dlib
from tqdm import tqdm
from google.colab import drive

drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [2]:
#make a virtual environment to downgrade python and tensorflow
!apt-get install python3-venv
!python3.6 -m venv /content/deep3d
!source /content/deep3d/bin/activate; pip install pillow argparse scipy matplotlib ipykernel
!source /content/deep3d/bin/activate; pip install --upgrade setuptools; pip install tensorflow-gpu==1.12.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-venv is already the newest version (3.6.7-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 69 not upgraded.
Requirement already up-to-date: setuptools in ./deep3d/lib/python3.6/site-packages


In [17]:
#downgrade cuda to 9.0
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda=9.0.176-1

--2021-05-19 03:14:04--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.16.29
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.16.29|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb [following]
--2021-05-19 03:14:04--  https://developer.nvidia.com/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?p1Nav48H6gINPwAHY5fXvd6o5kWx690VEMsUGPWNktUbd_D3YXbIslRj4ww1zkwnRuYRZ-FQ8wcXO5elx07mDXHeY9-Fb5K

In [5]:
!git clone https://github.com/gtangg12/Text2Video.git
!git clone https://github.com/microsoft/Deep3DFaceReconstruction.git

Cloning into 'Text2Video'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 91 (delta 41), reused 55 (delta 15), pack-reused 0
Unpacking objects: 100% (91/91), done.
Checking out files: 100% (11/11), done.
Cloning into 'Deep3DFaceReconstruction'...
remote: Enumerating objects: 420, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 420 (delta 0), reused 1 (delta 0), pack-reused 417
Receiving objects: 100% (420/420), 142.40 MiB | 24.29 MiB/s, done.
Resolving deltas: 100% (185/185), done.


In [6]:
%cd /content/Text2Video/
!git pull
%cd /content/

/content/Text2Video
Already up to date.
/content


In [37]:
#copy models and pretrained weights from google drive
!cp /content/drive/MyDrive/6869/01_MorphableModel.mat /content/Deep3DFaceReconstruction/BFM/
!cp /content/drive/MyDrive/6869/Exp_Pca.bin /content/Deep3DFaceReconstruction/BFM/
!mkdir /content/Deep3DFaceReconstruction/network/
!cp /content/drive/MyDrive/6869/FaceReconModel.pb /content/Deep3DFaceReconstruction/network/
!cp /content/drive/MyDrive/6869/rasterize_triangles_kernel.so /content/Deep3DFaceReconstruction/renderer/

#copy augmented code
!cp /content/drive/MyDrive/6869/demo.py /content/Deep3DFaceReconstruction/
!cp /content/drive/MyDrive/6869/face_decoder.py /content/Deep3DFaceReconstruction/
!cp /content/drive/MyDrive/6869/mesh_renderer.py /content/Deep3DFaceReconstruction/renderer/
!cp /content/drive/MyDrive/6869/rasterize_triangles.py /content/Deep3DFaceReconstruction/renderer/

#copy inputs and output directory
!cp /content/drive/MyDrive/6869/xAAmF3H0-ek.zip /content/Deep3DFaceReconstruction/
!mkdir /content/Deep3DFaceReconstruction/xAAmF3H0-ek/
!unzip /content/Deep3DFaceReconstruction/xAAmF3H0-ek.zip -d /content/Deep3DFaceReconstruction/
!cp /content/drive/MyDrive/6869/xAAmF3H0-ek_landmarks_pooled_fromdir.npy /content/Deep3DFaceReconstruction/
!mkdir /content/Deep3DFaceReconstruction/debug_outputs/
!mkdir /content/Deep3DFaceReconstruction/output/coeffs/

mkdir: cannot create directory ‘/content/Deep3DFaceReconstruction/network/’: File exists
mkdir: cannot create directory ‘/content/Deep3DFaceReconstruction/xAAmF3H0-ek/’: File exists
Archive:  /content/Deep3DFaceReconstruction/xAAmF3H0-ek.zip
replace /content/Deep3DFaceReconstruction/xAAmF3H0-ek/frame_0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: mkdir: cannot create directory ‘/content/Deep3DFaceReconstruction/debug_outputs/’: File exists


In [25]:
!ls /content/Deep3DFaceReconstruction/xAAmF3H0-ek/

frame_0.jpg	 frame_13858.jpg  frame_2289.jpg  frame_6145.jpg
frame_10000.jpg  frame_13859.jpg  frame_228.jpg   frame_6146.jpg
frame_10001.jpg  frame_1385.jpg   frame_2290.jpg  frame_6147.jpg
frame_10002.jpg  frame_13860.jpg  frame_2291.jpg  frame_6148.jpg
frame_10003.jpg  frame_13861.jpg  frame_2292.jpg  frame_6149.jpg
frame_10004.jpg  frame_13862.jpg  frame_2293.jpg  frame_614.jpg
frame_10005.jpg  frame_13863.jpg  frame_2294.jpg  frame_6150.jpg
frame_10006.jpg  frame_13864.jpg  frame_2295.jpg  frame_6151.jpg
frame_10007.jpg  frame_13865.jpg  frame_2296.jpg  frame_6152.jpg
frame_10008.jpg  frame_13866.jpg  frame_2297.jpg  frame_6153.jpg
frame_10009.jpg  frame_13867.jpg  frame_2298.jpg  frame_6154.jpg
frame_1000.jpg	 frame_13868.jpg  frame_2299.jpg  frame_6155.jpg
frame_10010.jpg  frame_13869.jpg  frame_229.jpg   frame_6156.jpg
frame_10011.jpg  frame_1386.jpg   frame_22.jpg	  frame_6157.jpg
frame_10012.jpg  frame_13870.jpg  frame_2300.jpg  frame_6158.jpg
frame_10013.jpg  frame_13871.jpg

In [5]:
%cd /content/Deep3DFaceReconstruction/
!source /content/deep3d/bin/activate; python demo.py

Streaming output truncated to the last 5000 lines.
   [-0.380548    0.39018217  0.22136095]
   [-0.37081498  0.3900781   0.22396974]
   [-0.36111122  0.39000013  0.2259224 ]]

  [[-0.40027922  0.38098866  0.21003805]
   [-0.3904813   0.38083977  0.21386524]
   [-0.38072455  0.38072455  0.21682538]
   [-0.37098417  0.3806201   0.21950862]
   [-0.36126935  0.3805371   0.22164342]]

  [[-0.40043065  0.3714839   0.20633432]
   [-0.39064828  0.37135702  0.20967838]
   [-0.38087744  0.371235    0.21289629]
   [-0.37113395  0.37113395  0.21555932]
   [-0.36140567  0.37104312  0.21795274]]

  [[-0.40061134  0.3619982   0.2019155 ]
   [-0.39080733  0.36185867  0.20569149]
   [-0.38102517  0.36173278  0.20909925]
   [-0.37126386  0.3616207   0.21213445]
   [-0.36151576  0.36151582  0.21497335]]]]
Using pre-trained .pb file.
2021-05-19 04:56:23.735129: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1511] Adding visible gpu devices: 0
2021-05-19 04:56:23.735212: I tensorflow/core/common_runtim

In [7]:
!zip -r /content/Deep3DFaceReconstruction/output.zip /content/Deep3DFaceReconstruction/output/coeffs/

updating: content/Deep3DFaceReconstruction/output/coeffs/ (stored 0%)
  adding: content/Deep3DFaceReconstruction/output/coeffs/rwcs_1962_150.npy (deflated 63%)
  adding: content/Deep3DFaceReconstruction/output/coeffs/masks_762_150.npy (deflated 99%)
  adding: content/Deep3DFaceReconstruction/output/coeffs/rwcs_16212_150.npy (deflated 63%)
  adding: content/Deep3DFaceReconstruction/output/coeffs/masks_1512_150.npy (deflated 99%)
  adding: content/Deep3DFaceReconstruction/output/coeffs/coeffs_15012_150.npy (deflated 6%)
  adding: content/Deep3DFaceReconstruction/output/coeffs/masks_3012_150.npy (deflated 99%)
  adding: content/Deep3DFaceReconstruction/output/coeffs/wcs_14712_150.npy (deflated 60%)
  adding: content/Deep3DFaceReconstruction/output/coeffs/coeffs_13212_150.npy (deflated 6%)
  adding: content/Deep3DFaceReconstruction/output/coeffs/wcs_6162_150.npy (deflated 60%)
  adding: content/Deep3DFaceReconstruction/output/coeffs/coeffs_7812_150.npy (deflated 6%)
  adding: content/Deep3

In [11]:
!du -h output.zip
#!cp output.zip /content/drive/MyDrive/6869/

7.4G	output.zip


In [45]:
ls /content/Deep3DFaceReconstruction/renderer/

camera_utils.py  mesh_renderer.py  rasterize_triangles_kernel.so
__init__.py      __pycache__/      rasterize_triangles.py


In [20]:
video_path = './drive/MyDrive/6869/xAAmF3H0-ek_video.mp4'
vid = cv2.VideoCapture(video_path)
print(vid.isOpened())
rate = vid.get(cv2.CAP_PROP_FPS)

True


In [52]:
landmarks = np.load('/content/drive/MyDrive/6869/xAAmF3H0-ek_landmarks_pooled_fromdir.npy', allow_pickle = True)
landmarks = landmarks.reshape(-1, 25, 2)
landmarks[17100]

array([[ 92., 120.],
       [139., 115.],
       [121., 150.],
       [106., 173.],
       [138., 170.],
       [105., 174.],
       [111., 170.],
       [117., 167.],
       [122., 168.],
       [127., 166.],
       [133., 167.],
       [139., 170.],
       [134., 174.],
       [129., 176.],
       [124., 177.],
       [119., 178.],
       [112., 177.],
       [108., 173.],
       [117., 171.],
       [123., 171.],
       [127., 170.],
       [137., 170.],
       [128., 170.],
       [123., 171.],
       [118., 171.]])

In [6]:
def frame_iter(capture, description):
    def _iterator():
        while capture.grab():
            yield capture.retrieve()[1]
    return tqdm(
        _iterator(),
        desc=description,
        total=int(capture.get(cv2.CAP_PROP_FRAME_COUNT)),
    )

In [7]:
from Text2Video.face_landmark_detector import landmarks_from_image, LIP_LANDMARKS, FIVE_LANDMARKS
all_5landmarks = []
for frame in frame_iter(vid, "move to dir"):
    frame = frame[:,:,::-1]
    try:
        landmarks = landmarks_from_image(frame, FIVE_LANDMARKS)
    except IndexError: #no faces
        landmarks = np.empty((5,2))
        landmarks[:] = np.nan
    all_5landmarks.append(landmarks)

move to dir:   0%|          | 32/17139 [00:23<3:24:38,  1.39it/s]

KeyboardInterrupt: ignored

In [ ]:
all_landmarks = []
lip_landmarks = range(49, 68)
for frame in frame_iter(vid, "landmark detection"):
    frame = frame[:,:,::-1] #change to RGB
    try:
        face = face_detector(frame, 1)[0]
        landmarks = landmark_detector(frame, face)
        landmarks_array = []
        for idx in lip_landmarks:
            landmarks_array.append([landmarks.part(idx).x, landmarks.part(idx).y])
        all_landmarks.append(landmarks_array)
    except IndexError:
        landmarks_array = np.empty((20,2))
        landmarks_array[:] = np.nan
        all_landmarks.append(landmarks_array)

In [ ]:
np.save(np.array('./drive/MyDrive/6869/landmarks_trump',all_landmarks))